# <center>EE2703 Applied Programming Lab</center>
## <center> Assignment 6 - Speeding Up with Cython</center>
### <center> EE23B102 - Sanjeev Subrahmaniyan</center>

In [1]:
# Essential imports for benchmarking and testing
import math
import cython
import numpy as np
from typing import Callable
from numpy.typing import NDArray

In [2]:
# Defining the number of trapezoids in an integral call
N = 1_000_000 #Set to 1 million iterations for inital comparison

In [3]:
# A function to quantify the errors from the numerical methods against the analytical solution
def percentage_error(true_value, estimated_value):
    try:
        error = abs(true_value - estimated_value) / abs(true_value) * 100
        return error
    except ZeroDivisionError:
        return "Zero division error"

## Task 1 - Vanilla python implementation of the trapezoidal rule of integration

For the first task, a vanilla python function which takes in the arguments of the integration(the limits, piece width and the function) is created. Complementary functions for x^2, sinx, e^x, and 1/x are also created with simple python definitions. These will be used for the comparison and benchmarking of the different implmentations.

In [4]:
def py_trapz(f: Callable[[float], float], a: float, b: float, n: int) -> float:
    """
    Evaluates the definite integral of a function using the trapezoidal rule.
    
    Parameters:
        f: Function to integrate
        a: Lower bound of integration
        b: Upper bound of integration
        n: Number of trapezoids to use
        
    Returns:
        float: Approximate value of the definite integral evaluted using the trapezoidal rule of integration
    """
    
    delx = (b - a) / n  # The width of the trapezoid
    area = 0.0
    for i in range(0, n):
        area += f(a + delx * i)  # Adding the trapezoidal area for the ith trapezoid
    area *= 2
    area -= (f(a) + f(b))  # Decrementing the double addition of edge points
    area *= (delx/2)  # Scaling the result to obtain the final area
    return area

def x2(x: float) -> float:
    """Computes x squared."""
    return x ** 2

def sinx(x: float) -> float:
    """Computes the sine of x."""
    return math.sin(x)

def ex(x: float) -> float:
    """Computes e raised to the power x."""
    return math.exp(x)

def xinv(x: float) -> float:
    """Computes the reciprocal of x (1/x)."""
    return 1/x

Knowing that the objective of cython is to speed up the programs, I can notice a few places in the previous code which could be potentially taking up significant execution time. For instance, the types of the variables are not forced, which would lead to python checking for validity and performing type casting when required. There are also cases where exceptions might need to be raised and checked for, like division by zero in the xinv function. Although typing hints might make the function better usable for the user, it does not make any difference for the python interpreter and would be significantly slowing down the execution of the function.

### Execution times for the vanilla implementation using timeit

In [5]:
%timeit py_trapz(x2, 0, 1, N)
%timeit py_trapz(sinx, 0, math.pi, N)
%timeit py_trapz(ex, 0, 1, N)
%timeit py_trapz(xinv, 1, 2, N)

161 ms ± 2.63 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
163 ms ± 4.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
151 ms ± 818 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
137 ms ± 5.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Accuracy of vanilla implementation

In [6]:
print(f"Error percentage for x^2 using vanilla method: {percentage_error(1/3, py_trapz(x2, 0, 1, N))}")
print(f"Error percentage for sinx using vanilla method: {percentage_error(2, py_trapz(sinx, 0, math.pi, N))}")
print(f"Error percentage for e^x using vanilla method: {percentage_error(math.e - 1, py_trapz(ex, 0, 1, N))}")
print(f"Error percentage for 1/x using vanilla method: {percentage_error(math.log(2), py_trapz(xinv, 1, 2, N))}")

Error percentage for x^2 using vanilla method: 0.0002999999498987105
Error percentage for sinx using vanilla method: 7.961409309586998e-11
Error percentage for e^x using vanilla method: 0.00015819766567642054
Error percentage for 1/x using vanilla method: 7.21347439658791e-05


## Task 2 - Numpy implementation

To set the highest standards for benchmarking, numpy is chosen due to its suitability for numerical calculations. The functions that were defined previously are redefined with numpy based types and formats. Note that all the numpy based functions start with np.

In [7]:
def np_trapz(f: Callable[[NDArray[np.float64]], NDArray[np.float64]], 
             a: float, b: float, n: int) -> float:
    """
    Integration using the trapezoidal rule and numpy support
    
    Parameters:
    f: The function for which the area is to be evaluated.
    a: The lower bound of the integration
    b: The upper bound of the integration
    n: The number of trapezoids used for the integration
    
    Returns:
    The area of the function obtained using the trapz numpy function."""
    step = (b - a)/n
    x = np.arange(a, b + step, step)
    y = f(x)
    return np.trapz(y, x, step)

def np_x2(x: NDArray[np.float64]) -> NDArray[np.float64]:
    """Vectorized x^2 function."""
    return np.square(x)

def np_ex(x: NDArray[np.float64]) -> NDArray[np.float64]:
    """Vectorized exponential function."""
    return np.exp(x)

def np_xinv(x: NDArray[np.float64]) -> NDArray[np.float64]:
    """Vectorized reciprocal function."""
    return 1/x

def np_sinx(x: NDArray[np.float64]) -> NDArray[np.float64]:
    """Vectorized sine function."""
    return np.sin(x)

To get a picture of the kind of execution speeds numpy offers, I run the same integrals with the functions defined as above. The expectation is that the speeds would be significantly higher due to how numpy is optimised for numericals and implemented in C.

In [8]:
%timeit np_trapz(np_x2, 0, 1, N)
%timeit np_trapz(np_sinx, 0, math.pi, N)
%timeit np_trapz(np_ex, 0, 1, N)
%timeit np_trapz(np_xinv, 1, 2, N)

13.9 ms ± 209 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
23.2 ms ± 242 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
19.7 ms ± 152 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
16.3 ms ± 838 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


As can be seen from the timeit results above and in the previous section, a simple numpy implementation of the integral function runs about 5-10 times faster on an average. This is a significant speedup.

### Accuracy of numpy implementation

In [9]:
print(f"Error percentage for x^2 using numpy method: {percentage_error(1/3, np_trapz(np_x2, 0, 1, N))}")
print(f"Error percentage for sinx using numpy method: {percentage_error(2, np_trapz(np_sinx, 0, math.pi, N))}")
print(f"Error percentage for e^x using numpy method: {percentage_error(math.e - 1, np_trapz(np_ex, 0, 1, N))}")
print(f"Error percentage for 1/x using numpy method: {percentage_error(math.log(2), np_trapz(np_xinv, 1, 2, N))}")

Error percentage for x^2 using numpy method: 5.0009996144240176e-11
Error percentage for sinx using numpy method: 8.223421943398534e-11
Error percentage for e^x using numpy method: 8.360843780518362e-12
Error percentage for 1/x using numpy method: 7.212880875335562e-05


## Task 3 - Cython Implementation

This section of the notebook demonstrates the use of Cython and the different levels of optimisation that can be achieved with it. Cython can significantly improve execution speeds when wisely used and implemented. I split the section into 2 parts which demonstrate the different levels of speedups for:
1. Simply compiling the program with Cython
1. Optimising the program fully with C data types, handling error checks and function returns.

In [10]:
%load_ext Cython 
## Loads the Cython extension for usage in the notebook

### Basic Cython implementation without optimisations
For the first part, I will implement the function simply using the Cython compiler and not add any other type optimisations. Note that the individual functions are also not changed and hence simply reused from the vanilla implemntation

In [11]:
%%cython --annotate

def cy_trapz1(f, a, b, n):
    delx = (b - a) / n
    area = 0
    for i in range(0, n):
        area += f(a + delx * i)
    area *= 2
    area -= (f(a) + f(b))
    area *= (delx/2)
    return area

As can be seen above, the yellow lines show locations where python interacts and performs checks and conversions and so on. The final objective would be to minimise those instances to achieve the maximum optimisation. The times of execution a level 1 optimisation can be seen for the same conditions as below. 

In [12]:
%timeit cy_trapz1(x2, 0, 1, N)
%timeit cy_trapz1(sinx, 0, math.pi, N)
%timeit cy_trapz1(ex, 0, 1, N)
%timeit cy_trapz1(xinv, 1, 2, N)

165 ms ± 7.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
162 ms ± 5.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
165 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
126 ms ± 4.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


As can be seen above, simply running the function with cython does not significantly speed up the execution(infact, it appears to be slowing things down). Why?

As it turns out, simply using a cython compiler might sometimes add more overheads than a python interpreter itself. This is particularly noticeable because no other optimisations have been performed. In the next part, we should be able to achieve a better performance than this.

### Optimised Cython implementation

Towards achieving a faster performance, the functions can be optimised by using C types in the function definitions. Further improvements like forcing and specifying the return types of the functions like C functions and excluding a few error checks(this would require the user to be careful with how the functions are used) can also be done. This is done by taking a look at the cython --annotate output for the previous implementation and spotting where the improvements can be made. The following block of code performs those functions with apt comments where changes are made:

In [13]:
%%cython --annotate

from libc.math cimport sin as csin, exp as cexp 
cimport cython

# The function types are defined as Enums to permit their usage in the function that will be implemented
# Not doing so will raise compatibility error as we will be attempting to run C functions in python without the proper method

cdef enum FuncType:
    X2 = 0
    EX = 2
    SINX = 1
    XINV = 3

# The following lines show the definition of the function in a C format
# This means the arguments and returns are statically typed and reduce the overhead of checking
# Some error checks have also been added to speed up to the full potential
@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
cdef float cy_x2(float x) :
    return x*x

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
cdef float cy_ex(float x) :
    return cexp(x)

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
cdef float cy_sinx(float x) :
    return csin(x)

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
cdef float cy_xinv(float x) :
    return 1/x

# This part of the cell shows the trapezoidal function
@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.binding(True)
# The function is defined using cpdef, which makes it a C type function with a python wrapper
# The function has static types for returns and arguments
cpdef double cy_trapz(int func_type, float a, float b, int n): 
    # The variables inside the program are defined as C type variables(statically typed)
    cdef double delx = (b - a) / n
    cdef double area = 0
    cdef float x
    cdef int i
    
    # Computing the trapezoidal areas based for the required function
    for i in range(n):
        x = a + delx * i
        if func_type == X2:
            area += cy_x2(x)
        elif func_type == EX:
            area += cy_ex(x)
        elif func_type == SINX:
            area += cy_sinx(x)
        elif func_type == XINV:
            area += cy_xinv(x)
            
    area *= 2 # Scaling the area as required
    
    # Subtracting the edge points to remove the double counting
    x = a
    if func_type == X2:
        area -= cy_x2(x)
    elif func_type == EX:
        area -= cy_ex(x)
    elif func_type == SINX:
        area -= cy_sinx(x)
    elif func_type == XINV:
        area -= cy_xinv(x)
        
    x = b
    if func_type == X2:
        area -= cy_x2(x)
    elif func_type == EX:
        area -= cy_ex(x)
    elif func_type == SINX:
        area -= cy_sinx(x)
    elif func_type == XINV:
        area -= cy_xinv(x)
        
    area *= (delx/2)
    return area

To gain an feel for the speed of the cython implementation, the functions are run with the same arguments as before and they are timed. Note that here, the functions themselves are also defined as c type functions and hence ofter phenomenal optimisation speeds.

In [14]:
%timeit cy_trapz(0, 0, 1, N)
%timeit cy_trapz(1, 0, math.pi, N)
%timeit cy_trapz(2, 0, 1, N)
%timeit cy_trapz(3, 1, 2, N)

1.39 ms ± 7.84 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
13.9 ms ± 83.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
10.7 ms ± 59.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.83 ms ± 70.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Evidently, the times taken for execution are much lesser than the vanilla python implementation. Interestingly, in some cases it is even faster than numpy itself, while for other cases it offers a similar but better performance.

### Error percentages for optimised cython implementation

In [15]:
print(f"Error percentage for x^2 using cython method: {percentage_error(1/3, cy_trapz(0, 0, 1, N))}")
print(f"Error percentage for sinx using cython method: {percentage_error(2, cy_trapz(1, 0, math.pi, N))}")
print(f"Error percentage for e^x using cython method: {percentage_error(math.e - 1, cy_trapz(2, 0, 1, N))}")
print(f"Error percentage for 1/x using cython method: {percentage_error(math.log(2), cy_trapz(3, 1, 2, N))}")

Error percentage for x^2 using cython method: 0.00030075873979629186
Error percentage for sinx using cython method: 7.707612326157687e-10
Error percentage for e^x using cython method: 0.00015859976785862772
Error percentage for 1/x using cython method: 7.231444627813809e-05


## Performance analysis

In [16]:
N = 10_000_000 #10 million trapezoids for the test

In [17]:
%timeit py_trapz(x2, 0, 10, N)
%timeit np_trapz(np_x2, 0, 10, N)
%timeit cy_trapz(0, 0, 10, N)

print(f"Error percentage for x^2 using vanilla method: {percentage_error(1/3, py_trapz(x2, 0, 1, N))}")
print(f"Error percentage for x^2 using numpy method: {percentage_error(1/3, np_trapz(np_x2, 0, 1, N))}")
print(f"Error percentage for x^2 using cython method: {percentage_error(1/3, cy_trapz(0, 0, 1, N))}")

1.74 s ± 34.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
150 ms ± 562 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
14.6 ms ± 169 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Error percentage for x^2 using vanilla method: 2.9999992140483656e-05
Error percentage for x^2 using numpy method: 3.0000003398145125e-05
Error percentage for x^2 using cython method: 2.6492704746639006e-05


## Conclusion

As can be seen above, using Cython smartly can offer performance and speed improvements of upto 100 times. While the accuracy is comparable for all the implementations, the Cython version's speed can prove to be of great use in computationally intensive tasks. However, converting python code to Cython code intelligently is often a meticulous process and hence does not scale quite well for large scale problems.